### Reference

1. https://github.com/ashishpatel26/LLM-Finetuning/blob/main/7.FineTune_LLAMA2_with_QLORA.ipynb  
2. https://huggingface.co/blog/4bit-transformers-bitsandbytes  
3. https://pytorch.org/blog/finetune-llms/  
4. https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing#scrollTo=6k_nL6xJMZW2  

# 1. Weight and Bias Login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_08"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

# 2. Login Huggingface

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 3. Dataset Load

In [3]:
from datasets import load_dataset

In [4]:
ko_instruction_01 = load_dataset("aeolian83/OpenOrca-gugugo-ko_test", cache_dir="/mnt/t7/.cache/huggingface/datasets")

In [5]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [6]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 1008132
    })
    test: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 1232162
    })
})

In [7]:
ko_instruction_01 = ko_instruction_01["train"]

In [8]:
ko_instruction_01

Dataset({
    features: ['question', 'system_prompt', 'response', 'id'],
    num_rows: 1008132
})

In [9]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.2)

In [10]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 806505
    })
    test: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 201627
    })
})

In [11]:
ko_instruction_01["train"][100]

{'question': '다음 질문에 답하세요: 1~5점 척도(1은 가장 불리한 점수, 5는 가장 유리한 점수)로 이 리뷰를 평가한다면? "안녕하세요, 이 제품을 사용해보고 싶습니다."\n답변:',
 'system_prompt': '당신은 인공지능 비서입니다. 사용자가 과제를 부여할 것입니다. 당신의 목표는 가능한 한 충실하게 과제를 완료하는 것입니다. 과제를 수행하는 동안 단계별로 생각하고 단계를 정당화하세요.',
 'response': '1\n\n단계별 프로세스:\n\n1. 리뷰를 이해합니다: 사용자가 작성한 리뷰 "안녕하세요, 이 제품을 사용해보고 싶습니다."는 실제로 리뷰가 아니라 제품이나 서비스를 사용해보고 싶다는 의사를 표현한 문구입니다. 리뷰에는 일반적으로 특정 제품이나 서비스를 사용한 후의 피드백이나 의견이 포함됩니다.\n\n2. 평가 기준: 일반적으로 1~5점 척도에서 5는 가장 긍정적이고 1은 가장 부정적이거나 가장 호의적이지 않은 평가입니다.\n\n3. 리뷰 분석: 사용자가 제품이나 서비스를 아직 사용해보지 않았기 때문에 사용자의 경험에 대한 긍정적 또는 부정적 피드백을 제공하지 않았기 때문에 평점을 부여할 수 없습니다. \n\n4. 결론: 따라서 해당 제품/서비스에 대한 사용자의 경험에 대한 구체적인 정보가 없기 때문에 가장 불리한 평점인 1점을 부여합니다. 그러나 이는 제품이나 서비스에 대한 부정적인 평가를 반영하는 것이 아니라 리뷰에 정보가 없는 콘텐츠가 부족하다는 것을 반영할 뿐입니다.',
 'id': 't0.263840'}

In [12]:
dataset_name = "aeolian83/OpenOrca-gugugo-ko_test"

# 4. Loading the Model

In [13]:
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [15]:
# Settings for 4-bit QLoRA Training(4bit QLoRA 학습을 위한 설정)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16, # Nvidia의 Ampere 아키텍처 이후 가속기는 bf16으로 속도 향상을 꾀할수 있다. 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# bnb_4bit_quant_type="nf4" 설정상 기본값은 bnb_4bit_quant_type="fp4"이나 허깅페이스 저자들에 의하면
# 경험적 결과로 "nf4"가 결과가 더 좋았다고 한다. https://huggingface.co/blog/4bit-transformers-bitsandbytes
# bnb_4bit_use_double_quant=True로 하면 매개변수단 0.4bit을 추가로 절약 할 수 있다고 한다. 

In [16]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=cache_model_dir, trust_remote_code=True)
model.config.use_cache = False

# model.config.pretraining_tp = 1
# 종종 QLoRA 코드에 이 코드가 보이는데 병렬 학습에 쓰이는 코드로 보인다. 

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_model_dir)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})



1

In [18]:
# 이 코드를 쓰지 않는 경우(물론 패딩 토큰을 별도로 사용하는 경우에 해당됨) loss가 0으로 떨어지는 경우가 있다함
tokenizer.padding_side = "right"

In [19]:
model.resize_token_embeddings(len(tokenizer)) # pad_token이 추가되었으므로 embedding과 language modeling head를 resize

Embedding(46337, 4096)

# 5. LoRA Setup

In [20]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [21]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# 6. Formatting Dataset

In [22]:
def format_instruction(sample):
    system_prompt = f"### system prompt: {sample['system_prompt']}"
    input = f"### question: {sample['question']}" if len(sample["question"]) > 0 else None
    output = f"### response: {sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [23]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/806505 [00:00<?, ? examples/s]

In [24]:
train_dataset["text"][100]

'### system prompt: 당신은 인공지능 비서입니다. 사용자가 과제를 부여할 것입니다. 당신의 목표는 가능한 한 충실하게 과제를 완료하는 것입니다. 과제를 수행하는 동안 단계별로 생각하고 단계를 정당화하세요.\n\n### question: 다음 질문에 답하세요: 1~5점 척도(1은 가장 불리한 점수, 5는 가장 유리한 점수)로 이 리뷰를 평가한다면? "안녕하세요, 이 제품을 사용해보고 싶습니다."\n답변:\n\n### response: 1\n\n단계별 프로세스:\n\n1. 리뷰를 이해합니다: 사용자가 작성한 리뷰 "안녕하세요, 이 제품을 사용해보고 싶습니다."는 실제로 리뷰가 아니라 제품이나 서비스를 사용해보고 싶다는 의사를 표현한 문구입니다. 리뷰에는 일반적으로 특정 제품이나 서비스를 사용한 후의 피드백이나 의견이 포함됩니다.\n\n2. 평가 기준: 일반적으로 1~5점 척도에서 5는 가장 긍정적이고 1은 가장 부정적이거나 가장 호의적이지 않은 평가입니다.\n\n3. 리뷰 분석: 사용자가 제품이나 서비스를 아직 사용해보지 않았기 때문에 사용자의 경험에 대한 긍정적 또는 부정적 피드백을 제공하지 않았기 때문에 평점을 부여할 수 없습니다. \n\n4. 결론: 따라서 해당 제품/서비스에 대한 사용자의 경험에 대한 구체적인 정보가 없기 때문에 가장 불리한 평점인 1점을 부여합니다. 그러나 이는 제품이나 서비스에 대한 부정적인 평가를 반영하는 것이 아니라 리뷰에 정보가 없는 콘텐츠가 부족하다는 것을 반영할 뿐입니다.</s>'

# 7. Training Argument Setup

In [25]:
from transformers import TrainingArguments

In [26]:
checkpoint_dir = "./checkpoint/gugugo-experi_01"

In [27]:
output_dir = checkpoint_dir
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
report_to="wandb"
save_steps = 500
save_total_limit=5
num_train_epochs = 0.9
logging_steps = 20
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [28]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [29]:
from trl import SFTTrainer

max_seq_length = 1024

In [30]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    # max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/806505 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [31]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# 8. Training

In [32]:
# trainer.train()

In [33]:
trainer.train(resume_from_checkpoint=True)

  0%|          | 0/362927 [00:00<?, ?it/s]

{'loss': 1.0349, 'grad_norm': 0.53125, 'learning_rate': 2.3524098182303667e-05, 'epoch': 0.8}
{'loss': 1.1966, 'grad_norm': 0.64453125, 'learning_rate': 2.3512735804839805e-05, 'epoch': 0.8}
{'loss': 1.2557, 'grad_norm': 0.51171875, 'learning_rate': 2.3501373427375943e-05, 'epoch': 0.8}
{'loss': 1.296, 'grad_norm': 0.640625, 'learning_rate': 2.3490011049912084e-05, 'epoch': 0.8}
{'loss': 1.0313, 'grad_norm': 0.7734375, 'learning_rate': 2.3478648672448226e-05, 'epoch': 0.8}
{'loss': 1.2645, 'grad_norm': 0.9140625, 'learning_rate': 2.3467286294984363e-05, 'epoch': 0.8}
{'loss': 1.1992, 'grad_norm': 1.203125, 'learning_rate': 2.3455923917520505e-05, 'epoch': 0.8}
{'loss': 1.1567, 'grad_norm': 0.462890625, 'learning_rate': 2.3444561540056643e-05, 'epoch': 0.8}
{'loss': 1.1183, 'grad_norm': 0.625, 'learning_rate': 2.343319916259278e-05, 'epoch': 0.8}
{'loss': 1.0736, 'grad_norm': 0.92578125, 'learning_rate': 2.342183678512892e-05, 'epoch': 0.8}
{'loss': 1.1836, 'grad_norm': 0.7265625, 'lear

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.4249, 'grad_norm': 0.734375, 'learning_rate': 2.324003874570715e-05, 'epoch': 0.8}
{'loss': 1.1613, 'grad_norm': 0.6640625, 'learning_rate': 2.3228676368243292e-05, 'epoch': 0.8}
{'loss': 1.2463, 'grad_norm': 0.3984375, 'learning_rate': 2.3217313990779434e-05, 'epoch': 0.8}
{'loss': 1.3271, 'grad_norm': 0.76953125, 'learning_rate': 2.320595161331557e-05, 'epoch': 0.8}
{'loss': 1.052, 'grad_norm': 0.9453125, 'learning_rate': 2.319458923585171e-05, 'epoch': 0.8}
{'loss': 1.3381, 'grad_norm': 0.484375, 'learning_rate': 2.318322685838785e-05, 'epoch': 0.8}
{'loss': 1.1833, 'grad_norm': 0.84375, 'learning_rate': 2.3171864480923992e-05, 'epoch': 0.8}
{'loss': 1.24, 'grad_norm': 0.51171875, 'learning_rate': 2.316050210346013e-05, 'epoch': 0.8}
{'loss': 1.1762, 'grad_norm': 0.57421875, 'learning_rate': 2.3149139725996267e-05, 'epoch': 0.8}
{'loss': 1.0722, 'grad_norm': 0.99609375, 'learning_rate': 2.313777734853241e-05, 'epoch': 0.8}
{'loss': 1.2813, 'grad_norm': 0.515625, 'learning

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


...

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.3042, 'grad_norm': 0.50390625, 'learning_rate': 5.152838179860754e-07, 'epoch': 0.9}
{'loss': 1.2142, 'grad_norm': 0.72265625, 'learning_rate': 5.039214405222149e-07, 'epoch': 0.9}
{'loss': 1.3196, 'grad_norm': 0.51953125, 'learning_rate': 4.925590630583543e-07, 'epoch': 0.9}
{'loss': 1.089, 'grad_norm': 0.5078125, 'learning_rate': 4.811966855944939e-07, 'epoch': 0.9}
{'loss': 1.0441, 'grad_norm': 1.34375, 'learning_rate': 4.698343081306332e-07, 'epoch': 0.9}
{'loss': 1.386, 'grad_norm': 0.515625, 'learning_rate': 4.584719306667727e-07, 'epoch': 0.9}
{'loss': 1.179, 'grad_norm': 0.828125, 'learning_rate': 4.471095532029122e-07, 'epoch': 0.9}
{'loss': 1.2427, 'grad_norm': 0.58984375, 'learning_rate': 4.357471757390517e-07, 'epoch': 0.9}
{'loss': 1.2359, 'grad_norm': 0.57421875, 'learning_rate': 4.2438479827519114e-07, 'epoch': 0.9}
{'loss': 1.024, 'grad_norm': 1.34375, 'learning_rate': 4.1302242081133053e-07, 'epoch': 0.9}
{'loss': 1.3356, 'grad_norm': 0.64453125, 'learning_r

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.2982, 'grad_norm': 0.65625, 'learning_rate': 2.3122438138956198e-07, 'epoch': 0.9}
{'loss': 1.1463, 'grad_norm': 0.69921875, 'learning_rate': 2.198620039257014e-07, 'epoch': 0.9}
{'loss': 1.1995, 'grad_norm': 0.490234375, 'learning_rate': 2.084996264618409e-07, 'epoch': 0.9}
{'loss': 1.0831, 'grad_norm': 0.59765625, 'learning_rate': 1.9713724899798036e-07, 'epoch': 0.9}
{'loss': 1.0388, 'grad_norm': 0.91796875, 'learning_rate': 1.857748715341198e-07, 'epoch': 0.9}
{'loss': 1.3806, 'grad_norm': 0.46875, 'learning_rate': 1.7441249407025927e-07, 'epoch': 0.9}
{'loss': 1.1799, 'grad_norm': 0.6640625, 'learning_rate': 1.6305011660639871e-07, 'epoch': 0.9}
{'loss': 1.1103, 'grad_norm': 0.60546875, 'learning_rate': 1.516877391425382e-07, 'epoch': 0.9}
{'loss': 1.1816, 'grad_norm': 0.7265625, 'learning_rate': 1.4032536167867765e-07, 'epoch': 0.9}
{'loss': 0.9829, 'grad_norm': 1.0625, 'learning_rate': 1.2896298421481712e-07, 'epoch': 0.9}
{'loss': 1.1696, 'grad_norm': 0.625, 'learnin

TrainOutput(global_step=362927, training_loss=0.1352296082772137, metrics={'train_runtime': 34616.5275, 'train_samples_per_second': 20.968, 'train_steps_per_second': 10.484, 'train_loss': 0.1352296082772137, 'epoch': 0.9})

In [34]:
lora_model_save_dir = "./results/gugugo-experi_01"

In [35]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(lora_model_save_dir, save_embedding_layers = True)

In [36]:
lora_config = LoraConfig.from_pretrained(lora_model_save_dir)
model = get_peft_model(model, lora_config)

In [37]:
tokenizer.push_to_hub('aeolian83/llama_ko_sft_gugugo_experi_01')

CommitInfo(commit_url='https://huggingface.co/aeolian83/llama_ko_sft_gugugo_experi_01/commit/19dd71bb9c3aebf4c5be4ad2c4a15d34a7a999d6', commit_message='Upload tokenizer', commit_description='', oid='19dd71bb9c3aebf4c5be4ad2c4a15d34a7a999d6', pr_url=None, pr_revision=None, pr_num=None)